In [9]:
from opdynamics.simulation import initialize_model
from opdynamics.utils.model_modification import modify_model

In [21]:
params = {
    "graph_type": "barabasi",
    "network_size": 500,
    "memory_size": 160,
    "code_length": 5,
    "kappa": 30,
    "lambd": 0,
    "alpha": 0,
    "omega": 0, 
    "gamma": 1,
    "prefferential_attachment": 2,
    "polarization_grouping_type": 1,
    "T": 500,
    "num_repetitions": int,
    "results_path": "results"
}

model = initialize_model(**params)
# model = modify_model(model)

In [22]:
from opdynamics.utils.types import Parameters

def param_to_hash(params: Parameters) -> str:
    param_tuple = tuple(params.values())
    return str(hash(param_tuple))

In [23]:
param_to_hash(params)

'-1252619067306615623'

In [31]:
f = open("results/last_run.txt", "w")
f.write("0")
f.close()

AttributeError: 'int' object has no attribute 'close'

In [30]:
f = open("results/last_run.txt", "r")
int(f.read())

0